# Gradient Centralization for Better Training Performance (勾配集中化による学習性能の向上)

**Author:** [Rishit Dagli](https://github.com/Rishit-dagli)<br>
**Date created:** 06/18/21<br>
**Last modified:** 06/18/21<br>
**Description:** Implement Gradient Centralization to improve training performance of DNNs.

## 序章

この例では、YongらによるDeep Neural Networksの新しい最適化手法である[Gradient Centralization](https://arxiv.org/abs/2004.01461)を実装し、
Laurence Moroneyの[Horses or Humansans]で実証しています。
Laurence Moroney氏の[Horses or Humans Dataset](https://www.tensorflow.org/datasets/catalog/horses_or_humans)で実証します。
勾配集中化することで、学習プロセスの高速化とDNNの最終的な汎化性能の向上の両方を実現できます。
これは、勾配のベクトルがゼロ平均になるように集中化することで、勾配を直接操作するものです。
勾配集中化はさらに、損失関数とその勾配のリプシッツ性を改善することで、学習プロセスをより効率的で安定したものとなります。

この例では、TensorFlow 2.2以降と、次のコマンドでインストールできる`tensorflow_datasets`が必要です。
下記コマンドでインストールすることができます。

```sh
pip install tensorflow-datasets
```

この例では、グラデーション集中化を実装していますが、私が作ったパッケージでも
私が作ったパッケージを使えば、簡単に使うことができます。
[gradient-centralization-tf](https://github.com/Rishit-dagli/Gradient-Centralization-TensorFlow)

## 設定

In [ ]:
from time import time

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

## データの準備

今回の例では、[Horses or Humans dataset](https://www.tensorflow.org/datasets/catalog/horses_or_humans)を使用します。

In [ ]:
num_classes = 2
input_shape = (300, 300, 3)
dataset_name = "horses_or_humans"
batch_size = 128
AUTOTUNE = tf.data.AUTOTUNE

(train_ds, test_ds), metadata = tfds.load(
    name=dataset_name,
    split=[tfds.Split.TRAIN, tfds.Split.TEST],
    with_info=True,
    as_supervised=True,
)

print(f"Image shape: {metadata.features['image'].shape}")
print(f"Training images: {metadata.splits['train'].num_examples}")
print(f"Test images: {metadata.splits['test'].num_examples}")

## データオーグメンテーションの活用

ここでは、データを `[0, 1]` にリスケールし、データに簡単な補強を施します。

In [ ]:
rescale = layers.Rescaling(1.0 / 255)

data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.3),
        layers.RandomZoom(0.2),
    ]
)


def prepare(ds, shuffle=False, augment=False):
    # Rescale dataset
    ds = ds.map(lambda x, y: (rescale(x), y), num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1024)

    # Batch dataset
    ds = ds.batch(batch_size)

    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(
            lambda x, y: (data_augmentation(x, training=True), y),
            num_parallel_calls=AUTOTUNE,
        )

    # Use buffered prefecting
    return ds.prefetch(buffer_size=AUTOTUNE)

データのリスケールとオーグメンテーション

In [ ]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
test_ds = prepare(test_ds)

## モデル定義

ここでは、Convolutional neural networkの定義を説明します。

In [ ]:
model = tf.keras.Sequential(
    [
        layers.Conv2D(16, (3, 3), activation="relu", input_shape=(300, 300, 3)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(32, (3, 3), activation="relu"),
        layers.Dropout(0.5),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.Dropout(0.5),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

## 勾配集中法の実装

今後はオプティマイザクラスである`RMSProp`をサブクラス化します。
クラスをサブクラス化し、`tf.keras.optimizers.Optimizer.get_gradients()`メソッドを変更して、勾配集中化しています。
簡単に説明すると、例えば、Double-Double-Doubleの逆伝播で勾配を得るとします。
密層や畳み込み層のバックプロポーゲーションによって勾配を得たとすると、次に重み行列の列ベクトルの平均を計算します。
重み行列の列ベクトルの平均を計算し、各列ベクトルから平均を取り除きます。

[本論文](https://arxiv.org/abs/2004.01461)では、様々なアプリケーションに関する実験を行っています。
一般的な画像分類、細分化された画像分類、検出とセグメンテーション、同一人物見地の実験では、GCがDNN学習の性能を一貫して向上させることができることを示しています。

また、現時点ではシンプルにするために、グラデーションのクリッピング機能は実装していません。
しかし、これは非常に簡単に実装できます。

現時点では、`RMSProp`オプティマイザーのサブクラスを作成しています。
しかし、他のオプティマイザーやカスタムオプティマイザーでも同じように簡単に再現できます。
オプティマイザーでも同じように再現できます。このクラスは、後のセクションで、次のように使用します。
勾配集中法でモデルを学習する際に使用します。

In [ ]:
class GCRMSprop(RMSprop):
    def get_gradients(self, loss, params):
        # We here just provide a modified get_gradients() function since we are
        # trying to just compute the centralized gradients.

        grads = []
        gradients = super().get_gradients()
        for grad in gradients:
            grad_len = len(grad.shape)
            if grad_len > 1:
                axis = list(range(grad_len - 1))
                grad -= tf.reduce_mean(grad, axis=axis, keep_dims=True)
            grads.append(grad)

        return grads


optimizer = GCRMSprop(learning_rate=1e-4)

## トレーニングユーティリティー

また、コールバックを作成して、トレーニングの合計時間と各エポックにかかった時間を簡単に測定できるようにします。
効果を比較することに興味があるからです。

In [ ]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time() - self.epoch_time_start)

## GCを使用しないモデルのトレーニング

次に、先ほど構築したモデルをGradient Centralizationなしでトレーニングします。
これを、Gradient Centralizationを用いて学習したモデルの学習性能と比較します。

In [ ]:
time_callback_no_gc = TimeHistory()
model.compile(
    loss="binary_crossentropy",
    optimizer=RMSprop(learning_rate=1e-4),
    metrics=["accuracy"],
)

model.summary()

また、履歴を保存しておくことで、勾配集中法で学習したモデルとそうでないモデルを後で比較することができます。

In [ ]:
history_no_gc = model.fit(
    train_ds, epochs=10, verbose=1, callbacks=[time_callback_no_gc]
)

## GCでモデルを学習

同じモデルを、今度はGradient Centralizationを使ってトレーニングします。
今回は、オプティマイザーがGradient Centralizationを使用していることに注目してください。

In [ ]:
time_callback_gc = TimeHistory()
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

model.summary()

history_gc = model.fit(train_ds, epochs=10, verbose=1, callbacks=[time_callback_gc])

## パフォーマンスの比較

In [ ]:
print("Not using Gradient Centralization")
print(f"Loss: {history_no_gc.history['loss'][-1]}")
print(f"Accuracy: {history_no_gc.history['accuracy'][-1]}")
print(f"Training Time: {sum(time_callback_no_gc.times)}")

print("Using Gradient Centralization")
print(f"Loss: {history_gc.history['loss'][-1]}")
print(f"Accuracy: {history_gc.history['accuracy'][-1]}")
print(f"Training Time: {sum(time_callback_gc.times)}")

読者の皆様には、様々な分野のデータセットでGradient Centralizationを試し、その効果を試すことをお勧めします。
その効果を試してみてください。また、[原著論文](https://arxiv.org/abs/2004.01461)をご覧になることを強くお勧めします。
Gradient Centralizationに関するいくつかの研究を紹介しています。
性能、汎用性、学習時間をいかに改善できるかを示していますし、より効率的です。

この実装をレビューしてくれた[Ali Mustufa Shaikh](https://github.com/ialimustufa)に感謝します。